In [1]:
import csv
import subprocess
import pandas as pd
import time

In [8]:
def execute_command(command):
    """Execute a given command in the shell and return its output."""
    count = 0
    while True:
        try:
            result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True, errors='ignore')
            print(f"Executed: {command}")
            return result.stdout.strip()
        except subprocess.CalledProcessError as e:
            print(f"An error occurred: {e}")
            # return f"Error: {str(e.stderr.strip())}"
            if count > 5:
                return 'Panicked'
            if 'PANIC' in str(e.stderr.strip()):
                time.sleep(8)
                count += 1
                print('TRYING AGAIN')
                continue
            if 'PANIC' not in str(e.stderr.strip()):
                return f"Error: {str(e.stderr.strip())}"

In [9]:
def get_python_file_path(id, slug):
    id = int(id)
    new_id = f"{id:04d}"
    return "python/" + new_id + "." + slug + "/solution.py"

In [10]:
def write_answer_to_file(answer, file_path):
    """Replaces the content between '# @lc code=begin' and '# @lc code=end' markers in the specified file with the provided answer."""
    # Read the existing content of the file
    with open(file_path, 'r') as file:
        content = file.readlines()
    
    # Initialize variables to hold the updated content and the state of being inside the code block
    updated_content = []
    in_code_block = False  # Tracks whether the current line is between the code markers
    
    # Iterate through each line of the file
    for line in content:
        if '# @lc code=begin' in line:
            in_code_block = True  # Set flag to True as we enter the code block
            updated_content.append(line)  # Append the begin marker
            updated_content.append(answer + '\n')  # Append the new answer
            continue
        
        if '# @lc code=end' in line and in_code_block:
            updated_content.append(line)  # Append the end marker
            in_code_block = False  # Reset the flag after appending the answer
            continue
        
        # If not inside a code block, copy the line as is to the updated content
        if not in_code_block:
            updated_content.append(line)
    
    # Write the updated content back to the file
    with open(file_path, 'w') as file:
        file.writelines(updated_content)

In [30]:
def process_csv_row(csv_file_path, count):
    """Process each row in the given CSV file."""
    # tests = []
    submissions = []
#     count = 0
    with open(csv_file_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for i, row in df.loc[count:].iterrows():
            id_value = row['id']
            answer = row['Python_Solutions']
            slug = row['slug']
            # if count ==8:
            #     break
            if id_value == '':
                break

            # Execute the leetgo pick command
            pick_command = f"leetgo pick {id_value}"
            execute_command(pick_command)

            python_file_path = get_python_file_path(id_value, slug)
            # Write the answer to the Python file
            write_answer_to_file(answer, python_file_path)

            # Execute the test and submit commands
            # test = execute_command("leetgo test last -L")
            # tests.append(test)
            time.sleep(5)
            submission = execute_command("leetgo submit last")
            if submission == 'Panicked':
                break
#             submissions.append(submission)
            
            print(count)
            df.loc[count, 'Submissions'] = submission
            df.to_csv("Final.csv", index=False)
            count += 1
            # print(tests)
            # print(submissions)
#     return submissions

In [31]:
csv_file_path = 'PythonSolutions_ExcludingSkipsRev.csv'
df = pd.read_csv(csv_file_path)
process_csv_row(csv_file_path, count)

Executed: leetgo pick 1
An error occurred: Command 'leetgo submit last' returned non-zero exit status 1.
0
Executed: leetgo pick 3
An error occurred: Command 'leetgo submit last' returned non-zero exit status 1.
1
Executed: leetgo pick 4
An error occurred: Command 'leetgo submit last' returned non-zero exit status 1.
2
Executed: leetgo pick 5


KeyboardInterrupt: 

In [28]:
count=0

In [21]:
count

494